In [ ]:
# Imports
import neurods as nds
import numpy as np
import os
import matplotlib.pyplot as plt
import nibabel
import cortex
# Configure defaults for plots
plt.rcParams['image.interpolation'] = 'nearest'
plt.rcParams['image.aspect'] = 'auto'
plt.rcParams['image.cmap'] = 'viridis'
%matplotlib inline
from scipy.stats import zscore

Here are functions we implemented in class.

In [ ]:
from numpy.linalg import inv
def OLS(X,Y):
    return np.dot(inv(np.dot(X.T,X)),np.dot(X.T,Y))

def compute_correlation(matrix_1, matrix_2):
    matrix_1_norm  = zscore(matrix_1)
    matrix_2_norm  = zscore(matrix_2)
    corr = np.mean(matrix_1_norm*matrix_2_norm, axis = 0)
    return corr

# Complex Stimuli


We will use freely available data from the Mitchell 2008 science paper: https://www.cs.cmu.edu/afs/cs/project/theo-73/www/science2008/data.html

The experiment actually consist in subjects looking at words/line drawings that are presented in isolation:

<img src="figures/science.png" style="height: 300px;">


In [ ]:
# loading data:
basedir = os.path.join(nds.io.data_list['fmri'],'word_picture')
name = os.path.join(basedir,'subject_1.nii.gz')
volumes = nibabel.load(name)
data = volumes.get_data()
print(data.shape)
data = data.T
print(data.shape)

In [ ]:
#loading mask
name = os.path.join(basedir,'subject_1_mask.nii')
volume = nibabel.load(name)
mask = volume.get_data()
print(mask.shape)
mask = mask.T
print(mask.shape)

In [ ]:
# flatten data to a 2D matrix
data = data[:,mask>0]
print(data.shape)

# zscore the data
data = zscore(data, axis = 0)

In [ ]:
# this package allows us to work with matlab data, which we need here to load the variables
import scipy.io as sio

# here we load the 60 words that comprise our stimuli
words = sio.loadmat(os.path.join(basedir,'words.mat'))

words = [s[0][0] for s in words['words']]

print("Here are the stimulus words:\n")
print (" - ".join(words))

In this dataset, a stimulus was presented every 10 seconds, and the activity between 4 and 8 seconds after onset was averaged, resulting in one brain image for every stimulus presentation. Each stimulus was repeated 6 times, and the repetitions of all the stimuli was averaged.

In [ ]:
word_num = 38 # change the word number
sample_image = np.zeros_like(mask)-2
sample_image[mask>0] = data[word_num]
h = cortex.mosaic(sample_image) # can try with different color map: e.g. h = mosaic(image , cmap= cm.hot)
plt.title(words[word_num],size=30)

This dataset already accounts for the delay of the hemodynamic response, and therefore we should not be convolving our design matrix. We will see here how to contruct a design matrix appropriate for such an experiment.

How can we represent the activity for items that do not belong into clear conditions? 

We could try to make each word be a condition. Ending up with 60 conditions. We see each word only once. How would that help us? We would be able to compute a contrast map between "horse" and "table", but that would not tell us much about why these differences occur, as "horse" and "table" vary in many ways. Also, learning a response per word will not allow us to know what the activity will be like for new words, such as "goat", "pen" etc.

However, we know that new words have some features in common with our set of objects. What if we could learn the responses to specific properties of words (e.g. whether or not they are animate, whether or not they are edible etc...). Then we predict the activity of a novel word as a combination of the activities associated with its properties. For example, we can learn how the brain responds to objects that are manmade, inanimate, made of wood and that are used as tools, and we can estimate the brain response of "pen" as the combination of these responses.

We will do all this in the multivariate regression framework we have used in the last labs. 

First, we need an annotation of the properties of these words. From looking at the list of words, it's clear that there are many properties that different sets of them share.

We have access to a set of 218 questions for which every word has been labeled by multiple users on amazon mechanical Turk (Sudre et al., Neuroimage, 2012). These question were designed to represent the semantic properties of these objects. Additionally, 11 features describing the visual properties of the line drawings are also provided.

The scale of the features is 1-5 with 1 being a 100% no and 5 being 100% yes.

Try changing feature_i below. Try to see the different features, as well as the features 218-229 as well:

In [ ]:
feature_data = sio.loadmat(os.path.join(basedir,'features.mat'))

feature_names = feature_data['feature_names']
features = feature_data['features']
print("We have {0} features that describe the stimulus.\n".format(len(feature_names)))
#print feature_names

print("The features matrix therefore has {0} rows and {1}.\n".format(len(words),len(feature_names)))


feature_i = 10
print("FEATURE NUMBER {0}".format(feature_i))
print(feature_names[feature_i][0][0])
for i in range(15):
    print(words[i], features[i,feature_i])
    

In [ ]:
print ("Features 1 to 218\n")
for i in range(15):
    print (feature_names[i][0][0])
print ("...")

print ("\n\nFeatures 219 to 229\n")
for i in range(218,229):
    print (feature_names[i][0][0])

Let's take out two subsets of the features:

In [ ]:
features_1 = features[:,:59]
features_2 = features[:,218:]


## BUILDING A PREDICTIVE MODEL

### IT IS VERY IMPORTANT NOT TO USE TEST DATA IN TRAINING!!

To judge if a model has learned to predict brain activity outside, we need test it on data it has not seen in training. 

Imagine you have a small dataset with voxel responses to features, and some of the voxels have some noise that is correlated to one of the features. The probability of such an event becomes smaller as the dataset size increases, but at low sample sizes there is a good chance of finding spurious correlations. Such a correlation actually allows you to build a model that predicts brain activity from the features, but only in that dataset, since the noise is independent of the data and will not repeat in the same way in other datasets. However, for the voxels that show a real and strong enough response to the features, you will be able to learn a model that predicts brain activity from the features, and that model should generalize to new data.

This is why we always test a model on held out data that was not used in training. This allows us to judge whether the model is really predicting neural activity and not just fitted to noise in the sample.

Here we separate for you the words into a test and a train set:

In [ ]:
Test_index = [0,1,2,3,4,6,7,8,10,13,20,23]
Train_index = list(set(range(60)) - set(Test_index))

Train_X_1 = np.nan_to_num(zscore(features_1[Train_index,:]))
Train_X_2 = zscore(features_2[Train_index,:])
Train_Y = zscore(data[Train_index,:])

Test_X_1 = np.nan_to_num(zscore(features_1[Test_index,:]))
Test_X_2 = zscore(features_2[Test_index,:])
Test_Y = zscore(data[Test_index,:])


### Weight estimation and data prediction

We want to learn a function that predicts the activity for any word in terms of its features. 


#### Feature set 1
- Use the OLS function to estimate the brain response to the features in features_1 for every voxel.
- Use the estimated weights to predict the activity for the held-out words, using Test_X_1.
- Use the compute_correlation function to compute the correlation of your predicted activity and the real activity Test_Y
- Plot a flatmap of the prediction performance. Which regions are well predicted, why?

In [ ]:
# cor = compute_correlation(Test_Y, Pred_Y)
# vol = cortex.Volume(cor, 'MNI', 'atlas336', mask=mask, vmin=0, vmax=0.6, cmap='viridis')
# fig = cortex.quickflat.make_figure(vol, height=500)
# plt.title("prediction performance with features set 1", fontsize=20)

### STUDENT ANSWER


#### Feature set 2
Repeat the above for feature set 2:
- Use the OLS function to estimate the brain response to the features in features_2 for every voxel.
- Use the estimated weights to predict the activity for the held-out words, using Test_X_2.
- Use the compute_correlation function to compute the correlation of your predicted activity and the real activity Test_Y
- Plot a flatmap of the prediction performance. Which regions are well predicted, why?

In [ ]:
# cor = compute_correlation(Test_Y, Pred_Y)
# vol = cortex.Volume(cor, 'MNI', 'atlas336', mask=mask, vmin=0, vmax=1, cmap='viridis')
# fig = cortex.quickflat.make_figure(vol, height=500)
# plt.title("prediction performance with features set 2", fontsize=20)

### STUDENT ANSWER


Considering that features_1 has information about the object categories, and features_2 has information about the stimulus visual feature, how can you interpret the difference in the two maps?

In [ ]:
### STUDENT ANSWER
